In [1]:
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.messages.human import HumanMessage

from agential.cog.fewshots.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_REACT
from agential.cog.lats.node import Node
from agential.cog.lats.prompts import (
    HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    LATS_INSTRUCTION_HOTPOTQA,
    LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE, 
    LATS_VALUE_INSTRUCTION_HOTPOTQA,
)
from agential.cog.lats.strategies.qa import (
    LATSAmbigNQStrategy,
    LATSFEVERStrategy,
    LATSHotQAStrategy,
    LATSTriviaQAStrategy,
    parse_qa_action,
    parse_qa_value,
)
from agential.cog.react.output import ReActOutput
from agential.utils.docstore import DocstoreExplorer
from agential.cog.lats.agent import LATSAgent

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
)

In [5]:
agent = LATSAgent(llm=llm, benchmark="hotpotqa", n_samples=2, depth_limit=5)

In [11]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"

best_node, out = agent.generate(
    question=question,
    key=key,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_HOTPOTQA,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    value_prompt=LATS_VALUE_INSTRUCTION_HOTPOTQA,
    additional_keys={},
    reflect_additional_keys={},
    value_additional_keys={},
    max_iterations=1,
    reset=True
)

<PROMPT AGENT=====================================================================>
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the last passage successfully found by Search.
(3) Finish[answer], which returns the answer and finishes the task.

Here are some examples:
Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episode of m

In [12]:
out

[LATSOutput(iteration=0, current_node={'state': ReActOutput(thought='', action_type='', query='', observation='', answer='', external_tool_info={}), 'visits': 0, 'value': 0, 'depth': 0, 'is_terminal': False, 'reward': 0}, children_nodes=[{'state': ReActOutput(thought='I need to search for VIVA Media AG and find out its new acronym after changing its name in 2004.', action_type='Search', query='VIVA Media AG', observation="Could not find [VIVA Media AG]. Similar: ['MTV Music (Polish TV channel)', 'VIVA Plus', 'Paramount International Networks', 'VIVA (German TV channel)', 'Viacom (1952–2005)', 'Vauxhall Viva', 'Sartorius AG', 'GfK Entertainment charts', 'Spellbound Entertainment', 'Ag-gag']", answer='', external_tool_info={'search_result': "Could not find [VIVA Media AG]. Similar: ['MTV Music (Polish TV channel)', 'VIVA Plus', 'Paramount International Networks', 'VIVA (German TV channel)', 'Viacom (1952–2005)', 'Vauxhall Viva', 'Sartorius AG', 'GfK Entertainment charts', 'Spellbound Ent

In [13]:
best_node.state

ReActOutput(thought="Since direct searches for VIVA Media AG and its new acronym after the name change in 2004 did not provide relevant information, I should consider looking for industry reports, press releases, or official announcements related to the company's rebranding to uncover the acronym.", action_type='Search', query='VIVA Media AG rebranding press release', observation="Could not find [VIVA Media AG rebranding press release]. Similar: ['Virgin Interactive Entertainment', '2024 in Philippine television', 'Lake Las Vegas', 'Voice of America', 'Renault', 'PolyGram', 'Veolia Transport', 'British Midland International', 'Universal Music Group']", answer='', external_tool_info={'search_result': "Could not find [VIVA Media AG rebranding press release]. Similar: ['Virgin Interactive Entertainment', '2024 in Philippine television', 'Lake Las Vegas', 'Voice of America', 'Renault', 'PolyGram', 'Veolia Transport', 'British Midland International', 'Universal Music Group']", 'lookup_resul

: 